In [91]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [92]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [93]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

In [94]:
range_detector = RangeDriftDetector()
range_detector._get_features_range(X=y)

(np.float64(0.0001238776427896493), np.float64(0.9999334202177776))

In [95]:
range_detector.fit(series=series_dict)
range_detector.series_values_range_

{'series_1': (np.float64(0.0001039141050391823),
  np.float64(0.9999684161569717)),
 'series_2': (np.float64(8.295374594935456e-05),
  np.float64(0.9999148224569071))}

In [96]:
range_detector.fit(series=series_dict, exog=exog)
range_detector.series_values_range_
pprint(range_detector.exog_values_range_)

{'temp': (np.float64(0.00022439938284513072), np.float64(0.9999564532008017))}


# FIT

In [97]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(0.0001238776427896493), np.float64(0.9999334202177776))}
['y']


In [98]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001039141050391823),
              np.float64(0.9999684161569717)),
 'series_2': (np.float64(8.295374594935456e-05),
              np.float64(0.9999148224569071))}
['series_1', 'series_2']


In [99]:
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001039141050391823), np.float64(0.9999684161569717)), 'series_2': (np.float64(8.295374594935456e-05), np.float64(0.9999148224569071))} 
Exogenous value ranges: {} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: [] 
Series data type: <class 'pandas.core.frame.DataFrame'> 
Exogenous data type: None 

In [100]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001039141050391823),
              np.float64(0.9999684161569717)),
 'series_2': (np.float64(8.295374594935456e-05),
              np.float64(0.9999148224569071))}
['series_1', 'series_2']


In [101]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001039141050391823),
              np.float64(0.9999684161569717)),
 'series_2': (np.float64(8.295374594935456e-05),
              np.float64(0.9999148224569071))}
['series_1', 'series_2']


In [102]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.00022439938284513072), np.float64(0.9999564532008017))}
['temp']


In [103]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'B', 'A'},
 'cat_feature_1': {'C', 'B', 'A'},
 'cat_feature_2': {'C', 'B', 'A'},
 'cat_feature_3': {'C', 'B', 'A'},
 'cat_feature_4': {'C', 'B', 'A'},
 'feature_0': (np.float64(2.5287805849538714e-05),
               np.float64(0.9995583828726486)),
 'feature_1': (np.float64(0.0001190060853658581),
               np.float64(0.9996926284631035)),
 'feature_2': (np.float64(0.0002630111375639954),
               np.float64(0.9998489438573485)),
 'feature_3': (np.float64(2.165829618328008e-05),
               np.float64(0.9999131283796368)),
 'feature_4': (np.float64(0.0002988870357431006),
               np.float64(0.9997603224677335))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [104]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(2.5287805849538714e-05),
                            np.float64(0.9995583828726486)),
              'feature_1': (np.float64(0.0001190060853658581),
                            np.float64(0.9996926284631035)),
              'feature_2': (np.float64(0.0002630111375639954),
                            np.float64(0.9998489438573485)),
              'feature_3': (np.float64(2.165829618328008e-05),
                            np.float64(0.9999131283796368)),
              'feature_4': (np.float64(0.0002988870357431006),
                            np.float64(0.9997603224677335))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 

In [105]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 'A'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(2.5287805849538714e-05),
                            np.float64(0.9995583828726486)),
              'feature_1': (np.float64(0.0001190060853658581),
                            np.float64(0.9996926284631035)),
              'feature_2': (np.float64(0.0002630111375639954),
                            np.float64(0.9998489438573485)),
              'feature_3': (np.float64(2.165829618328008e-05),
                            np.float64(0.9999131283796368)),
              'feature_4': (np.float64(0.0002988870357431006),
                            np.float64(0.9997603224677335))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'C', 'B', 'A'},
              'cat_feature_2': {'C', 'B', 

# Predict

In [106]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(0.0001238776427896493), np.float64(0.9999334202177776))}
{'temp': (np.float64(0.00022439938284513072), np.float64(0.9999564532008017))}


In [107]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y': 'y' has one or more values outside the range seen during training [0.00012,     │
│ 0.99993]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :197                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [108]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
display(last_window_invalid_df)
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)

,series_1,series_2
2013-06-01,0.340091,0.912759
2013-06-02,10000.000000,10000.000000
2013-06-03,0.551928,0.628957
2013-06-04,0.675136,0.530819
2013-06-05,0.526000,0.397772
...,...,...
2013-09-04,0.843448,0.080683
2013-09-05,0.863935,0.213484
2013-09-06,0.206247,0.141483
2013-09-07,0.595660,0.379206


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'series_1' has one or more values outside the range seen during training │
│ [0.00010, 0.99997]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :197                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'series_2' has one or more values outside the range seen during training │
│ [0.00008, 0.99991]. This may affect the accuracy of the predictions.                 │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :197                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [109]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)

In [110]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

UnboundLocalError: cannot access local variable 'rule' where it is not associated with a value

In [111]:
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

TypeError: RangeDriftDetector.check_features_range_() got an unexpected keyword argument 'input_name'

In [112]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

TypeError: RangeDriftDetector.check_features_range_() got an unexpected keyword argument 'input_name'

In [113]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

TypeError: RangeDriftDetector.check_features_range_() got an unexpected keyword argument 'input_name'

In [114]:
range_detector = RangeDriftDetector()
range_detector.fit(series=series_dict, exog=exog_dict)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

TypeError: RangeDriftDetector.check_features_range_() got an unexpected keyword argument 'input_name'